In [154]:
import rasterio
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from pathlib import Path
from tqdm import tqdm
from skimage.restoration import denoise_tv_bregman
import sys
sys.path.append('utils/')
from rasterio.transform import array_bounds
from shapely.geometry import box
import geopandas as gpd
from osgeo import gdal
import re
import subprocess

### Import data into list (data)

In [155]:
data = Path('data/tandemx_mekong/').glob('*/*.tif')
data = sorted(data, key=lambda x: x.name)
data

[PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N08E104_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N08E105_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N08E106_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N09E103_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N09E104_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N09E105_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N09E106_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N10E103_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N10E104_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N10E105_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N10E106_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N11E103_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N11E104_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30_N11E105_DEM.tif'),
 PosixPath('data/tandemx_mekong/dem/TDM1_DEM__30

In [156]:
#putting the data into a string
strdata = str(data)
strdata
strdata1=strdata.replace("('","<")
strdata2=strdata1.replace("tif","tif>")
strdata2

"[PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N08E104_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N08E105_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N08E106_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N09E103_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N09E104_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N09E105_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N09E106_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N10E103_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N10E104_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N10E105_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N10E106_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N11E103_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N11E104_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N11E105_DEM.tif>'), PosixPath<data/tandemx_mekong/dem/TDM1_DEM__30_N11E106_DEM.t

In [157]:
data_new = re.findall(r'<(.+?)>',strdata2)
data_new

['data/tandemx_mekong/dem/TDM1_DEM__30_N08E104_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N08E105_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N08E106_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N09E103_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N09E104_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N09E105_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N09E106_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N10E103_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N10E104_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N10E105_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N10E106_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N11E103_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N11E104_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N11E105_DEM.tif',
 'data/tandemx_mekong/dem/TDM1_DEM__30_N11E106_DEM.tif']

In [158]:
in_directory = "/users/dewihenry/jpl/python/data/tandemx_mekong/dem"
files_to_process = glob.glob(os.path.join(in_directory, '*.tif'))

In [159]:
files_to_process

['/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E106_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E106_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E104_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E105_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E103_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N10E103_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E105_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E104_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N10E106_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E106_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E104_DEM.tif',
 '/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__3

In [160]:
files_string = " " .join(files_to_process)
print(files_string)

/users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E106_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E106_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E104_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E105_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E103_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N10E103_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N11E105_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N08E104_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N10E106_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E106_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N09E104_DEM.tif /users/dewihenry/jpl/python/data/tandemx_mekong/dem/TDM1_DEM__30_N10E105_DEM.tif /users/dewihenry/jpl/python/

this merges all the rasters together to make one single raster. kicking the function into a subterminal to execute gdal_merge

In [161]:
outfile = "out.tif"
cmd = "../../opt/anaconda3/bin/gdal_merge.py -o " + outfile + " " + files_string
!{cmd}

0...10...20...30...40...50...60...70...80...90...100 - done.


In [162]:
print (outfile)

out.tif


GDALWARP to convert from Ellipsoidal to Orthometric elevation:
https://spatialthoughts.com/2019/10/26/convert-between-orthometric-and-ellipsoidal-elevations-using-gdal/

In [164]:
outEGM = "mekong_EGM.tif"
cmd = "../../opt/anaconda3/bin/gdalwarp -s_srs EPSG:4326+4979 -t_srs EPSG:4326+5773 " + outfile + " " + outEGM
!{cmd} 

Processing out.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
